In [1]:
import pandas as pd
import math
import logging
import os
import uuid
import graphviz as gv
import itertools

In [2]:
logger = logging.getLogger('CART')
column_property_dict = ['is_discrete']

In [3]:
class Node(object):
    _uuid = None
    _feature = None
    _value = None
    _label = None
    _is_discrete = None
    _is_leaf = None
    _left_node = None
    _right_node = None

    @property
    def uuid(self):
        return self._uuid
    
    @property
    def label(self):
        return self._label
    
    @label.setter
    def label(self, value):
        self._label = value

    @property
    def feature(self):
        return self._feature
    
    @feature.setter
    def feature(self, value):
        self._feature = value

    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        self._value = value

    @property
    def is_leaf(self):
        return self._is_leaf
    
    @is_leaf.setter
    def is_leaf(self, value):
        self._is_leaf = value
    
    @property
    def is_discrete(self):
        return self._is_discrete
    
    @is_discrete.setter
    def is_discrete(self, value):
        self._is_discrete = value
    
    @property
    def left_node(self):
        return self._left_node
    
    @left_node.setter
    def left_node(self, value):
        self._left_node = value

    @property
    def right_node(self):
        return self._right_node
    
    @right_node.setter
    def right_node(self, value):
        self._right_node = value
        
    def _check_arg(self, key, kwargs):
        if key in kwargs:
            return kwargs[key]
        else:
            return None
    
    def __init__(self, **kwargs):
        self._uuid = uuid.uuid4()
        self._feature = self._check_arg('feature', kwargs)
        self._is_discrete = self._check_arg('is_discrete', kwargs)
        self._is_leaf = self._check_arg('is_leaf', kwargs)
        self._label = self._check_arg('label', kwargs)
        self._value = self._check_arg('value', kwargs)
    
    def print_node(self):
        if self._is_leaf:
            logger.info('Label Node [%s]' % (self._label))
        else:
            logger.info('Decision Node [%s], Value [%s]' % (self._feature, str(self._value)))
            self._left_node.print_node()
            self._right_node.print_node()
    
    def to_dot(self):
        edges = []
        if self._is_leaf:
            return []
        else:
            if self._is_discrete == True:
                if self._left_node.is_leaf:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="in %s"];' % (self._feature, self._uuid, self._left_node.label, self._left_node.uuid, self._value[0]))
                else:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="in %s"];' % (self._feature, self._uuid, self._left_node.feature, self._left_node.uuid, self._value[0]))
                edges.extend(self._left_node.to_dot())
                
                if self._right_node.is_leaf:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="in %s"];' % (self._feature, self._uuid, self._right_node.label, self._right_node.uuid, self._value[1]))
                else:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="in %s"];' % (self._feature, self._uuid, self._right_node.feature, self._right_node.uuid, self._value[1]))
                edges.extend(self._right_node.to_dot())
            else:
                if self._left_node.is_leaf:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="<= %s"];' % (self._feature, self._uuid, self._left_node.label, self._left_node.uuid, str(self._value)))
                else:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="<= %s"];' % (self._feature, self._uuid, self._left_node.feature, self._left_node.uuid, str(self._value)))
                edges.extend(self._left_node.to_dot())
                
                if self._right_node.is_leaf:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="> %s"];' % (self._feature, self._uuid, self._right_node.label, self._right_node.uuid, str(self._value)))
                else:
                    edges.append('"%s\n[%s]" -> "%s\n[%s]"[label="> %s"];' % (self._feature, self._uuid, self._right_node.feature, self._right_node.uuid, str(self._value)))
                edges.extend(self._right_node.to_dot())
        return edges
    
    def find_next_node(self, dataset, r_idx):
        if self._is_leaf == True:
            return None
        else:
            data = dataset.data
            val = data[self._feature][r_idx]
            if self._is_discrete == True:
                if val in self._value[0]:
                    return self._left_node
                elif val in self._value[1]:
                    return self._right_node
                else:
                    pass
            else:
                # support continuous data
                if val <= self._value:
                    return self._left_node
                else:
                    return self._right_node

In [4]:
class Dataset(object):
    _data = None
    _column_properties = None
    
    def __init__(self, data, column_properties):
        self._data = data
        self._column_properties = column_properties
    
    def __len__(self):
        return len(self._data)
    
    @property
    def data(self):
        return self._data
    
    @data.setter
    def data(self, value):
        self._data = value
    
    @property
    def column_properties(self):
        return self._column_properties
    
    @column_properties.setter
    def column_properties(self, value):
        self._column_properties = value
    
    def column_property(self, key):
        return self._column_properties[key]

    def column_property_val(self, key, feature):
        return self._column_properties[key][feature]

In [5]:
def load_dataset():
    return load_file()

In [6]:
def load_file(path=os.getcwd(), file='discrete_data.xlsx'):
    data = pd.read_excel(os.path.join(path, file), sheet_name=0)
    
    col_data = pd.read_excel(os.path.join(path, file), sheet_name=1)
    # skip 'column_property_dict' and 'label' columns
    columns = col_data.columns[1:-1]
    column_properties = {}
    for r_idx, row in col_data.iterrows():
        config = {}
        for col in columns:
            config[col] = row[col]
        column_properties[column_property_dict[r_idx]] = config
    return Dataset(data, column_properties)

In [7]:
def load_iris(path=os.getcwd(), file='iris.data'):
    data = pd.read_csv(os.path.join(path, file))
    column_properties = {column_property_dict[0]: {'SepalLengthCm': False, 
                                         'SepalWidthCm': False, 
                                         'PetalLengthCm': False, 
                                         'PetalWidthCm': False, 
                                         'Species': True}}
    return Dataset(data, column_properties)

In [8]:
def load_test_dataset():
    data = pd.DataFrame({'年龄':[25, 37], 
                         '年龄分类':['青年', '中年'], 
                         '有工作':['是', '是'], 
                         '有房子':['是', '是'], 
                         '信贷情况':['一般', '好'], 
                         '类别':['通过', '通过']})
    column_properties = {column_property_dict[0]: {'年龄': False, 
                                          '年龄分类': True, 
                                          '有工作': True, 
                                          '有房子': True, 
                                          '信贷情况': True, 
                                          '类别': True}}
    
    return Dataset(data, column_properties)

In [9]:
def choose_best_feature(dataset):
    data = dataset.data
    columns = data.columns[:-1]
    label = data.columns[-1]
    m = len(data)
    logging.debug('data:\n%s' % data)
    
    gini = 1.
    best_feature = None
    best_feature_val = None

    for feature in columns:
        gini_temp = 0.
        
        if dataset.column_property_val(column_property_dict[0], feature) == True:
            # support discrete data
            groups = feature_split_groups(dataset, feature)
            for group in groups:
                gini_temp = calc_group_gini(dataset, feature, group)
                if gini_temp < gini:
                    gini = gini_temp
                    best_feature = feature
                    best_feature_val = group
        else:
            # support continuous data
            sorted_data = data.sort_values(feature, ascending=True)
            unique_feature_vals = sorted_data[feature].unique()
            if len(unique_feature_vals) < 2:
                # c_entropy is zero, maxium info gain
                return feature, unique_feature_vals[0]
            elif len(unique_feature_vals) == 2:
                discrete_points = [(unique_feature_vals[0] + unique_feature_vals[1]) / 2]
            else:
                discrete_points = (unique_feature_vals[0:-2] + unique_feature_vals[1:-1]) / 2
                
            for pt in discrete_points:
                gini_temp = 0.
                l_gini_temp = 1.
                l_sub_data_temp, r_sub_data_temp = split_data(dataset, feature, pt)
                
                m_l_sub = len(l_sub_data_temp)
                for _, cnt in l_sub_data_temp.data[label].value_counts().items():
                    l_gini_temp = l_gini_temp - math.pow(cnt / m_l_sub, 2)
                gini_temp = gini_temp + m_l_sub / m * l_gini_temp
                
                r_gini_temp = 1.
                m_r_sub = len(r_sub_data_temp)
                for _, cnt in r_sub_data_temp.data[label].value_counts().items():
                    r_gini_temp = r_gini_temp - math.pow(cnt / m_r_sub, 2)
                gini_temp = gini_temp + m_r_sub / m * r_gini_temp
                
                if gini_temp < gini:
                    gini = gini_temp
                    best_feature = feature
                    best_feature_val = pt
    
    logging.info('best feature: %s, gini: %.6f' % (best_feature, gini))
    
    return best_feature, best_feature_val

In [10]:
def split_data(dataset, feature, value):
    data = dataset.data
    l_sub_data_temp = None
    r_sub_data_temp = None
    if dataset.column_property_val(column_property_dict[0], feature) == True:
        # type(value) = tuple(tuple(), tuple())
        l_sub_data_temp = data[data[feature].isin(value[0])].copy()
        r_sub_data_temp = data[data[feature].isin(value[1])].copy()
    else:
        # type(value) = float
        l_sub_data_temp = data[data[feature] <= value].copy()
        r_sub_data_temp = data[data[feature] > value].copy()
    
    l_sub_data_temp.drop(feature, axis=1, inplace=True)
    r_sub_data_temp.drop(feature, axis=1, inplace=True)
    
    return Dataset(l_sub_data_temp, dataset.column_properties), Dataset(r_sub_data_temp, dataset.column_properties)

In [11]:
def major_label(labels):
    value_counts = labels.value_counts().sort_values(ascending=False)
    logging.info('label conflict exists:\n%s' % (value_counts))
    return value_counts.index[0]

In [12]:
def create_tree(dataset):
    data = dataset.data
    labels = data[data.columns[-1]]
    unique_labels = labels.unique()
    
    # if only one unique label left, just pick this label
    if len(unique_labels) == 1:
        return Node(label=unique_labels[0], is_leaf=True)
    
    # if some conflicts on labels with same input, pick the label with highest probability
    if len(data.columns) == 1:
        return Node(label=major_label(labels), is_leaf=True)
    
    best_feature, best_feature_val = choose_best_feature(dataset)
    node = Node(feature=best_feature, value=best_feature_val, is_discrete=dataset.column_property_val(column_property_dict[0], best_feature), is_leaf=False)
                
    l_sub_data, r_sub_data = split_data(dataset, best_feature, best_feature_val)
    node.left_node = create_tree(l_sub_data)
    node.right_node = create_tree(r_sub_data)
        
    return node

In [13]:
def rating(model, dataset):
    data = dataset.data
    labels = data[data.columns[-1]]
    m = len(data)
    
    pred_data = pd.DataFrame()
    for r_idx, _ in data.iterrows():
        node = model
        while True:
            if node.is_leaf:
                pred_data = pred_data.append(pd.DataFrame({'pred_labels': [node.label]}), ignore_index=True)
                break
            node = node.find_next_node(dataset, r_idx)
    
    results = pred_data[pred_data['pred_labels'] == labels]
    rating = len(results) / m
    return rating

In [14]:
def feature_split_groups(dataset, feature):
    data = dataset.data
    uni_vals = data[feature].unique()
    m = len(uni_vals)
    c = []
    for i in range(1, m):
        c.extend(itertools.combinations(uni_vals, i))
    c_m = len(c)

    return zip(c[:int(c_m/2)], c[:int(c_m/2-1):-1])

In [15]:
def calc_group_gini(dataset, feature, group):
    data = dataset.data
    m = len(data)
    
    # len(group) = 2; D1, D2
    gini = [1.] * len(group)
    for i in range(len(group)):
        d = data[data[feature].isin(group[i])]
        unique_d_labels = d[data.columns[-1]].unique()

        for _, cnt in d[data.columns[-1]].value_counts().items():
            gini[i] = gini[i] - math.pow(cnt / len(d), 2)
        gini[i] = len(d) / m * gini[i]
    
    return math.fsum(gini)

In [16]:
def gen_graph(root):
    edge_data = ''
    edges = root.to_dot()
    for edge in edges:
        edge_data += edge
    dot_data = 'digraph edge_settings {%s}' % edge_data
    graph = gv.Source(dot_data)
    graph.render()
    logging.info('generate graph')

In [17]:
if __name__ == '__main__':
    # logging setting
#     logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')
    logging.basicConfig(level=logging.INFO, format='%(filename)s[%(lineno)d] - %(message)s')
    
    
    # model train
#     dataset = load_dataset()
    dataset = load_iris()
    
    root = create_tree(dataset)
    root.print_node()

    # graph
#     gen_graph(root)
    
    # verify
    verify_data = dataset
    verify_ratings = rating(root, verify_data)
    logging.info('verify rating: %.2f%%' % (verify_ratings * 100))
    
    # test
#     test_data = load_test_dataset()
#     ratings = fit(root, test_data)
    
#     logging.debug('test_data:\n%s' % test_data.data)
#     logging.info('rating: %.2f%%' % (ratings * 100))

<ipython-input-9-120d79b36342>[57] - best feature: PetalLengthCm, gini: 0.333333
<ipython-input-9-120d79b36342>[57] - best feature: PetalWidthCm, gini: 0.110306
<ipython-input-9-120d79b36342>[57] - best feature: SepalLengthCm, gini: 0.155271
<ipython-input-9-120d79b36342>[57] - best feature: SepalWidthCm, gini: 0.000000
<ipython-input-9-120d79b36342>[57] - best feature: SepalWidthCm, gini: 0.137019
<ipython-input-11-d5fddcdfd3b5>[3] - label conflict exists:
Iris-versicolor    3
Iris-virginica     1
Name: Species, dtype: int64
<ipython-input-11-d5fddcdfd3b5>[3] - label conflict exists:
Iris-versicolor    45
Iris-virginica      3
Name: Species, dtype: int64
<ipython-input-9-120d79b36342>[57] - best feature: SepalLengthCm, gini: 0.037267
<ipython-input-9-120d79b36342>[57] - best feature: SepalWidthCm, gini: 0.142857
<ipython-input-11-d5fddcdfd3b5>[3] - label conflict exists:
Iris-virginica     1
Iris-versicolor    1
Name: Species, dtype: int64
<ipython-input-3-60ee5b9aeba1>[89] - Decision